# Variables

In [114]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [115]:
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['/home/codespace/.python/current/lib/python312.zip', '/home/codespace/.python/current/lib/python3.12', '/home/codespace/.python/current/lib/python3.12/lib-dynload', '', '/home/codespace/.local/lib/python3.12/site-packages', '/home/codespace/.python/current/lib/python3.12/site-packages', '..', '..', '..']


# Testing

In [116]:
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [117]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\r\nNhận dạng chữ viết ta...",Link
1,2,Bối cảnh của vấn đề:\r\nTrong thời đại thông t...,Link
2,3,Bối cảnh của vấn đề:\r\nĐây là một nhiệm vụ hồ...,Link
3,4,Bối cảnh của vấn đề:\r\nĐiều quan trọng là các...,Link
4,5,Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng t...,Link
5,6,🐾 Story: Alice’s Magical Pet Photo Box\r\nAlic...,Link


In [118]:
task = df_task["Task"][4]

In [119]:
task

'Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng thực tế như tổng hợp nội dung hình ảnh, phân loại album, hay trích xuất đặc trưng nội dung cho ảnh tổ hợp, cần xác định chủ đề chính của một ảnh ghép từ nhiều ảnh con. Bài toán này đòi hỏi ứng dụng không chỉ nhận diện được nội dung của từng ảnh con, mà còn cần xác định chủ đề chiếm ưu thế trong toàn bộ ảnh tổng thể.\r\n\r\nYêu cầu cụ thể cần đạt được:\r\nMỗi đầu vào là một ảnh tổ hợp, ghép từ 9 ảnh màu nhỏ (3x3).\r\nNhiệm vụ của ứng dụng là:\r\nXác định chủ đề chiếm số lượng nhiều nhất trong 9 ảnh con.\r\nTrường hợp có nhiều chủ đề xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó làm kết quả.\r\n\r\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\r\nMột thư mục có tên là "images" chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) (3 hàng × 3 cột).\r\n\r\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\r\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\r\ncó các cột:\r\nfile_name: tên

In [120]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [121]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [122]:
class WebScraperParameters(BaseModel):
    url: str

In [123]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [124]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [125]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0.3
)

In [126]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )
    subtask_four: str = Field(
        ...,
        description="Detailed description of task 4"
    )

In [127]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [128]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
        You are an expert in task formulation. Given a machine learning task, you will
        help divide the task into these subtasks:
        - Subtask1: for identifying task description and classify the problem type.
        - Subtask2: for identifying the problem requirements as described in the "Yêu cầu cụ thể cần đạt được" (or other equivalent) section, including: problem requirements, expected input and expected output.
        - Subtask3: for identifying the input format of the problem, as being described in the "Định dạng dữ liệu đầu vào cho bài toán tổng thể" (or other equivalent) section.
        - Subtask4: for identifying the output format of the problem with the given classes, as being described in the "Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể" (or other equivalent) section.

        You **MUST** follow these guidelines:
        - Do not mistake the output classes with the one described in the problem requirements. The output classes **MUST** contain the classes described in the "Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể" section.
        - Describe the subtasks as specific as possible.
        """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [129]:
state = {
    "task": HumanMessage(content=task),
}

In [130]:
state = classify_agent(state)

In [131]:
pprint.pprint(state['subtasks'].subtask_one)

('Xác định bài toán và phân loại loại bài toán. Bài toán này là một bài toán '
 'phân loại ảnh, cụ thể là xác định chủ đề chiếm ưu thế trong một ảnh ghép từ '
 'nhiều ảnh con.')


In [132]:
pprint.pprint(state['subtasks'].subtask_two)

('Xác định yêu cầu của bài toán. Yêu cầu của bài toán là xác định chủ đề chiếm '
 'số lượng nhiều nhất trong 9 ảnh con của một ảnh tổ hợp. Nếu có nhiều chủ đề '
 'xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó '
 'làm kết quả. Đầu vào là một ảnh tổ hợp ghép từ 9 ảnh màu nhỏ (3x3). Đầu ra '
 'là một nhãn duy nhất đại diện cho chủ đề xuất hiện nhiều nhất trong 9 ảnh '
 'con.')


In [133]:
pprint.pprint(state['subtasks'].subtask_three)

('Xác định định dạng dữ liệu đầu vào. Dữ liệu đầu vào là một thư mục có tên là '
 "'images' chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) "
 '(3 hàng ×3 cột).')


In [134]:
pprint.pprint(state['subtasks'].subtask_four)

("Xác định định dạng kết quả đầu ra. Kết quả đầu ra là một file 'output.csv' "
 "với mỗi hàng là kết quả dự đoán mỗi ảnh, có các cột: 'file_name' (tên file "
 "ảnh) và 'prediction' (nhãn của ảnh). Nhãn của ảnh là một nhãn duy nhất đại "
 'diện cho chủ đề xuất hiện nhiều nhất trong 9 ảnh con. Các chủ đề có thể là '
 "một trong các nhãn sau: ['forest', 'orchid', 'spider', 'fox', 'caterpillar', "
 "'crocodile', 'whale', 'road', 'motorcycle', 'tiger', 'woman', 'orange', "
 "'elephant', 'otter', 'flatfish', 'cup', 'sea', 'mountain', 'cloud', "
 "'willow_tree', 'rabbit', 'couch', 'maple_tree', 'wardrobe', 'mushroom', "
 "'plain', 'telephone', 'shrew', 'bowl', 'tractor', 'camel', 'bus', 'cattle', "
 "'house', 'cockroach', 'lawn_mower', 'oak_tree', 'apple', 'bicycle', "
 "'beetle', 'palm_tree', 'sunflower', 'poppy', 'keyboard', 'porcupine', "
 "'snake', 'can', 'pear', 'shark', 'chimpanzee', 'hamster', 'train', 'cra', "
 "'bear', 'sweet_pepper', 'pine_tree', 'bed', 'tank', 'beaver', 'bridge', "

# Modeling Agent

In [135]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [136]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [137]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [138]:
res = model_selection_agent(state)

/tmp/ipykernel_30944/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_30944/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [139]:
pprint.pprint(res.models)

[ModelInfo(model='Phân loại hình ảnh', url='https://huggingface.co/zhaospei/Model_7'),
 ModelInfo(model='Phân loại hình ảnh thành 100 lớp đối tượng chi tiết (fine-grained)', url='https://huggingface.co/zhaospei/Model_6'),
 ModelInfo(model='Phân loại hình ảnh giống chó', url='https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit')]


In [140]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [141]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...

Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/zhaospei/Model_7'}`
responded: To solve this task, I need to analyze the provided models and select the best one that fits the given task description. The task involves determining the dominant theme in a collage image made of 9 smaller images.

First, I will use the `web_scrapper` function to gather detailed information about each model. This will help in understanding their capabilities and determining which one is best suited for the task.

I will start by scraping information about the first model:




🖼️ Mô hình Phân loại Ảnh - ViT Fine-tuned trên CIFAR-10
📝 Mô tả
Đây là mô hình Vision Transformer (ViT) được fine-tuned từ mô hình khác trên tập dữ liệu CIFAR-10. Mô hình được huấn luyện để phân loại ảnh vào 10 lớp khác nhau, mỗi lớp đại diện cho một danh mục đối tượng cụ thể.
📌 Nhiệm vụ
Loại bài toán: Phân loại ảnh (Image Classification)
Số lớp: 10 (Tương ứng nhãn) CIFAR-10
📥 Đầu 

In [142]:
pprint.pprint(res2['output'])

('To solve this task, I need to analyze the provided models and select the '
 'best one that fits the given task description. The task involves determining '
 'the dominant theme in a collage image made of 9 smaller images.\n'
 '\n'
 'I have already gathered detailed information about the three models:\n'
 '\n'
 '**Model 1: Phân loại hình ảnh — https://huggingface.co/zhaospei/Model_7**\n'
 '- Description: Vision Transformer (ViT) fine-tuned on CIFAR-10 dataset for '
 'image classification into 10 classes.\n'
 '- Input format: RGB images of size 224x224 pixels.\n'
 '- Output format: Probabilities for each class (logits) in a tensor of size '
 '[batch_size, 10].\n'
 '- Library requirements: transformers, torch, torchvision.\n'
 '- Example code: Provided in the tool output.\n'
 '\n'
 '**Model 2: Phân loại hình ảnh thành100 lớp đối tượng chi tiết (fine-grained) '
 '— https://huggingface.co/zhaospei/Model_6**\n'
 '- Description: Vision Transformer (ViT) fine-tuned on CIFAR-100 dataset for '

## Output Parser Agent

In [143]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

In [144]:
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

In [145]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [146]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [147]:
ans = output_parser_agent(state)

In [148]:
pprint.pprint(res2['output'])

('To solve this task, I need to analyze the provided models and select the '
 'best one that fits the given task description. The task involves determining '
 'the dominant theme in a collage image made of 9 smaller images.\n'
 '\n'
 'I have already gathered detailed information about the three models:\n'
 '\n'
 '**Model 1: Phân loại hình ảnh — https://huggingface.co/zhaospei/Model_7**\n'
 '- Description: Vision Transformer (ViT) fine-tuned on CIFAR-10 dataset for '
 'image classification into 10 classes.\n'
 '- Input format: RGB images of size 224x224 pixels.\n'
 '- Output format: Probabilities for each class (logits) in a tensor of size '
 '[batch_size, 10].\n'
 '- Library requirements: transformers, torch, torchvision.\n'
 '- Example code: Provided in the tool output.\n'
 '\n'
 '**Model 2: Phân loại hình ảnh thành100 lớp đối tượng chi tiết (fine-grained) '
 '— https://huggingface.co/zhaospei/Model_6**\n'
 '- Description: Vision Transformer (ViT) fine-tuned on CIFAR-100 dataset for '

In [149]:
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_requirements)
pprint.pprint(ans.model_sample_code)

('This is a Vision Transformer (ViT) model fine-tuned from the original '
 'google/vit-base-patch16-224-in21k on the CIFAR-100 dataset. The model is '
 'trained to classify images into 100 different classes, each representing a '
 'specific object category.')
'RGB images of size 224x224 pixels.'
'Probabilities for each class (logits) in a tensor of size [batch_size,100].'
('Install required libraries by running: pip install transformers torch '
 'torchvision')
('import torch\n'
 'from transformers import ViTForImageClassification, ViTImageProcessor\n'
 'from PIL import Image\n'
 '\n'
 '# Load image to classify\n'
 'image = Image.open("path_to_your_image.jpg")\n'
 '\n'
 '# Load processor and model from Hugging Face\n'
 'processor = ViTImageProcessor.from_pretrained("zhaospei/Model_6")\n'
 'model = ViTForImageClassification.from_pretrained("zhaospei/Model_6")\n'
 '\n'
 '# Process input\n'
 'inputs = processor(images=image, return_tensors="pt")\n'
 '\n'
 '# Predict with model\n'
 'with to

In [150]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


In [151]:
class AgentCodingState(TypedDict):
    problem_description: str | None
    problem_input_description: str | None
    problem_output_description: str | None
    model_description: str | None
    model_input_format: str | None
    model_output_format: str | None
    model_requirements: str | None
    model_sample_code: str | None
    output_classes: str | None
    code: str | None


In [152]:
base_prompt = r"""
    You are a specialist in Machine Learning. Your task is to generate a *fully functional with all necessary imports and dependencies* codebase in *Python* that can be executed flawlessly.

    You will be provided with:
    - A problem description
    - An input specification
    - An output specification
    - A model description

    ### Input:
    - Problem description: {problem_description}
    - Problem input specification: {problem_input_description}
    - Problem output specification: {problem_output_description}
    - ML model description: {model_description}
    - ML model input format: {model_input_format}
    - ML model output format: {model_output_format}
    - ML model requirements: {model_requirements}
    - ML model sample code: {model_sample_code}
    - Output classes: {output_classes}

    ### Guidelines

    You *must* strictly follow the following guidelines:
    - The preprocessing step should be suitable for the data type.
    - The postprocessing step should notices the differences between the data returned by the model and the output requirements. You must extract and use the exact class labels as defined in the output specification.
    - Do *not invent new labels or translate* the class names. Use them exactly as given.
    - You *must* make sure that your codebase can be executed flawlessly that would not encounter any errors or exceptions.
    - You must add some tqdm to see the infer progress.

    Your implementation *must strictly follow* the structure below:
    1. *Imports*: All required libraries.
    2. *Preprocessing*: Handle and transform the input as defined.
    3. *Inference logic*: Use the described model for prediction. You *must* use tqdm or similar logging library to track progress.
    4. *Postprocessing*: Format or transform the raw output into the final result as described.

    You must *not* include any explanations, markdown, or logging outside what is required by the problem.

    Return *only* the complete Python codebase, and you **MUST NOT** include a main function in any way. Wrap it with:
    \`\`\`python
    # code here
    \`\`\`
    """


In [153]:
class OutputState(TypedDict):
    output_classes: str | None

In [154]:
def output_agent(state: OutputState) -> OutputState:
    prompt = r"""
        Your are a specialist in machine learning. Your task is to identify the absolute classes of the given problem description, following with an output description.
        ### Input:
        - Problem description: {problem_description}
        - Output description: {output_description}

        You must return an array, strictly following these guidelines:
        - Understand the context from the given problem description.
        - Extract the class names from the output description. You **must not** invent new labels or translate the class names. Use them exactly as given in the output description.
        - Create an array containing the classes.

        You must return only the array containing those classes, without any formatting.
    """

    prompt = prompt.format(
        problem_description = state["problem_description"],
        output_description = state["output_description"]
    )
    response = llm.invoke(prompt)
    return {**state, "output_classes": response.content}

In [155]:
output_state = {
    "problem_description": task,
    "output_description": state["subtasks"].subtask_four
}

In [156]:
new_state = output_agent(output_state)

In [157]:
new_state["output_classes"]

"['forest', 'orchid', 'spider', 'fox', 'caterpillar', 'crocodile', 'whale', 'road', 'motorcycle', 'tiger', 'woman', 'orange', 'elephant', 'otter', 'flatfish', 'cup', 'sea', 'mountain', 'cloud', 'willow_tree', 'rabbit', 'couch', 'maple_tree', 'wardrobe', 'mushroom', 'plain', 'telephone', 'shrew', 'bowl', 'tractor', 'camel', 'bus', 'cattle', 'house', 'cockroach', 'lawn_mower', 'oak_tree', 'apple', 'bicycle', 'beetle', 'palm_tree', 'sunflower', 'poppy', 'keyboard', 'porcupine', 'snake', 'can', 'pear', 'shark', 'chimpanzee', 'hamster', 'train', 'cra', 'bear', 'sweet_pepper', 'pine_tree', 'bed', 'tank', 'beaver', 'bridge', 'girl', 'skyscraper', 'wolf', 'raccoon', 'rose', 'worm', 'table', 'pickup_truck', 'turtle', 'lizard', 'leopard', 'streetcar', 'rocket', 'man', 'tulip', 'lamp', 'seal', 'lion', 'butterfly', 'kangaroo', 'television', 'ray', 'chair', 'skunk', 'trout', 'possum', 'mouse']"

In [158]:
def coding_agent(state: AgentCodingState) -> AgentCodingState:
    prompt = base_prompt.format(
        problem_description=state["problem_description"],
        problem_input_description=state["problem_input_description"],
        problem_output_description=state["problem_output_description"],
        model_description=state["model_description"],
        model_input_format=state["model_input_format"],
        model_output_format=state["model_output_format"],
        model_requirements=state["model_requirements"],
        model_sample_code=state["model_sample_code"],
        output_classes=state["output_classes"],
    )
    response = llm.invoke(prompt)
    return {**state, "code": response.content}

In [159]:
prev_state = {
    "problem_description": task,
    "problem_input_description": state["subtasks"].subtask_three,
    "problem_output_description": state["subtasks"].subtask_four,
    "model_description": res2['output'],
    "model_input_format": ans.model_input_format,
    "model_output_format": ans.model_output_format,
    "model_requirements": ans.model_requirements,
    "model_sample_code": ans.model_sample_code,
    "output_classes": new_state["output_classes"]
}


In [160]:
coding_state = coding_agent(prev_state)

In [161]:
print(coding_state["code"])

```python
import os
import torch
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import csv
from tqdm import tqdm

# Load model and processor
processor = ViTImageProcessor.from_pretrained("zhaospei/Model_6")
model = ViTForImageClassification.from_pretrained("zhaospei/Model_6")

# Define output classes
output_classes = ['forest', 'orchid', 'spider', 'fox', 'caterpillar', 'crocodile', 
                  'whale', 'road', 'motorcycle', 'tiger', 'woman', 'orange', 
                  'elephant', 'otter', 'flatfish', 'cup', 'sea', 'mountain', 'cloud', 
                  'willow_tree', 'rabbit', 'couch', 'maple_tree', 'wardrobe', 
                  'mushroom', 'plain', 'telephone', 'shrew', 'bowl', 'tractor', 
                  'camel', 'bus', 'cattle', 'house', 'cockroach', 'lawn_mower', 
                  'oak_tree', 'apple', 'bicycle', 'beetle', 'palm_tree', 'sunflower', 
                  'poppy', 'keyboard', 'porcupine', 'snake', 'can', 'pear', 

In [162]:
coding_state

{'problem_description': 'Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng thực tế như tổng hợp nội dung hình ảnh, phân loại album, hay trích xuất đặc trưng nội dung cho ảnh tổ hợp, cần xác định chủ đề chính của một ảnh ghép từ nhiều ảnh con. Bài toán này đòi hỏi ứng dụng không chỉ nhận diện được nội dung của từng ảnh con, mà còn cần xác định chủ đề chiếm ưu thế trong toàn bộ ảnh tổng thể.\r\n\r\nYêu cầu cụ thể cần đạt được:\r\nMỗi đầu vào là một ảnh tổ hợp, ghép từ 9 ảnh màu nhỏ (3x3).\r\nNhiệm vụ của ứng dụng là:\r\nXác định chủ đề chiếm số lượng nhiều nhất trong 9 ảnh con.\r\nTrường hợp có nhiều chủ đề xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó làm kết quả.\r\n\r\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\r\nMột thư mục có tên là "images" chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) (3 hàng × 3 cột).\r\n\r\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\r\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\r\ncó cá

In [163]:
import os
import shutil
import subprocess
import tempfile
import venv
from pathlib import Path

def code_execute_agent(
    input_path,
    output_path,
    code: str,
    requirements=None,
    timeout: int = 120000
):
    requirements = requirements or []

    input_path = Path(input_path)
    output_path = Path(output_path)

    temp_dir = Path(tempfile.mkdtemp(prefix="code_runner_", dir=Path.cwd()))
    venv_dir = temp_dir / "venv"
    project_dir = temp_dir / "project"

    try:
        # Step 1: Create project structure
        os.makedirs(project_dir, exist_ok=True)

        # Step 2: Copy input to temp folder
        # if input_path.exists():
        #     if input_path.is_dir():
        #         shutil.copytree(input_path, project_dir / "input", dirs_exist_ok=True)
        #     else:
        #         shutil.copy2(input_path, project_dir / input_path.name)

        if input_path.exists():
            if input_path.is_dir():
                dest = project_dir / input_path.name
                shutil.copytree(input_path, dest, dirs_exist_ok=True)
            else:
                shutil.copy2(input_path, project_dir / input_path.name)

        # Step 3: Write code into main.py
        main_file = project_dir / "main.py"
        main_file.write_text(code)

        # Step 4: Setup virtual environment
        venv.create(venv_dir, with_pip=True)
        pip_path = venv_dir / "bin" / "pip"
        subprocess.check_call([str(pip_path), "install", *requirements])

        # # Step 5: Execute the code
        # python_path = venv_dir / "bin" / "python"
        # result = subprocess.run(
        #     [str(python_path), "main.py"],
        #     cwd=project_dir,
        #     stdout=subprocess.PIPE,
        #     stderr=subprocess.STDOUT,
        #     capture_output=True,
        #     text=True,
        #     timeout=timeout
        # )

        # if result.returncode != 0:
        #     raise RuntimeError(f"[Error]\n{result.stderr}")

        # Step 5: Execute the code with realtime logging
        python_path = venv_dir / "bin" / "python"
        process = subprocess.Popen(
            [str(python_path), "main.py"],
            cwd=project_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        log_lines = []
        try:
            for line in process.stdout:
                print(line, end="")              # Realtime print
                log_lines.append(line)           # Collect for return
            process.wait(timeout=timeout)
        except subprocess.TimeoutExpired:
            process.kill()
            raise RuntimeError("Code execution timed out.")

        if process.returncode != 0:
            raise RuntimeError(f"Execution failed:\n{''.join(log_lines)}")

        # Step 6: Copy specific output file to output_path
        produced_file = project_dir / output_path.name
        if not produced_file.exists():
            raise FileNotFoundError(f"Expected output file not found: {produced_file}")

        # Ensure parent folder for output_path exists
        output_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(produced_file, output_path)

        return result.stdout

    finally:
        # Step 7: Cleanup temp folder
        shutil.rmtree(temp_dir, ignore_errors=True)


In [164]:
# !gdown 1sHOzN7zdg3RDu8ijCXJXrL4KpYi4J_Vs
# !gdown 1uUqW8itSiO_RniR-eFDFu1VaTLgOhQ8l

In [165]:
coding_state

{'problem_description': 'Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng thực tế như tổng hợp nội dung hình ảnh, phân loại album, hay trích xuất đặc trưng nội dung cho ảnh tổ hợp, cần xác định chủ đề chính của một ảnh ghép từ nhiều ảnh con. Bài toán này đòi hỏi ứng dụng không chỉ nhận diện được nội dung của từng ảnh con, mà còn cần xác định chủ đề chiếm ưu thế trong toàn bộ ảnh tổng thể.\r\n\r\nYêu cầu cụ thể cần đạt được:\r\nMỗi đầu vào là một ảnh tổ hợp, ghép từ 9 ảnh màu nhỏ (3x3).\r\nNhiệm vụ của ứng dụng là:\r\nXác định chủ đề chiếm số lượng nhiều nhất trong 9 ảnh con.\r\nTrường hợp có nhiều chủ đề xuất hiện với tần suất bằng nhau, có thể chọn một chủ đề bất kỳ trong số đó làm kết quả.\r\n\r\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\r\nMột thư mục có tên là "images" chứa các ảnh màu RGB (288x288), được ghép từ 9 ảnh RGB con (32x32) (3 hàng × 3 cột).\r\n\r\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\r\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\r\ncó cá

In [166]:
coding_state["code"]

'```python\nimport os\nimport torch\nfrom transformers import ViTForImageClassification, ViTImageProcessor\nfrom PIL import Image\nimport csv\nfrom tqdm import tqdm\n\n# Load model and processor\nprocessor = ViTImageProcessor.from_pretrained("zhaospei/Model_6")\nmodel = ViTForImageClassification.from_pretrained("zhaospei/Model_6")\n\n# Define output classes\noutput_classes = [\'forest\', \'orchid\', \'spider\', \'fox\', \'caterpillar\', \'crocodile\', \n                  \'whale\', \'road\', \'motorcycle\', \'tiger\', \'woman\', \'orange\', \n                  \'elephant\', \'otter\', \'flatfish\', \'cup\', \'sea\', \'mountain\', \'cloud\', \n                  \'willow_tree\', \'rabbit\', \'couch\', \'maple_tree\', \'wardrobe\', \n                  \'mushroom\', \'plain\', \'telephone\', \'shrew\', \'bowl\', \'tractor\', \n                  \'camel\', \'bus\', \'cattle\', \'house\', \'cockroach\', \'lawn_mower\', \n                  \'oak_tree\', \'apple\', \'bicycle\', \'beetle\', \'p

In [167]:
print(sys.path)

['/home/codespace/.python/current/lib/python312.zip', '/home/codespace/.python/current/lib/python3.12', '/home/codespace/.python/current/lib/python3.12/lib-dynload', '', '/home/codespace/.local/lib/python3.12/site-packages', '/home/codespace/.python/current/lib/python3.12/site-packages', '..', '..', '..']


In [168]:
code_execute_agent(
    "images",
    "output.csv",
    coding_state["code"].replace("```python", "").replace("```", ""),
    ["torch", "huggingface", "huggingface_hub", "tqdm", "pandas", "numpy", "safetensors", "transformers", "pillow"],
    1200000
)

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/56/7e/67c3fe2b8c33f40af06326a3d6ae7776b3e3a01daa8f71d125d78594d874/torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Obtaining dependency information for huggingface from https://files.pythonhosted.org/packages/f4/8c/e61fbc39c0a37140e1d4941c4af29e2d53bacf9f4559e3de24d8f4e484f0/huggingface-0.0.1-py3-none-any.whl.metadata
  Using cached huggingface-0.0.1-py3-none-any.whl.metadata (2.9 kB)
  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/33/fb/53587a89fbc00799e4179796f51b3ad713c5de6bb680b2becb6d37c94649/huggingface_hub-0.33.0-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762a


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: /workspaces/ISE-challenge/Test/code_runner_9cyxo5ti/venv/bin/python3 -m pip install --upgrade pip



  0%|          | 0/201 [00:02<?, ?it/s]
Traceback (most recent call last):
  File "/workspaces/ISE-challenge/Test/code_runner_9cyxo5ti/project/main.py", line 63, in <module>
    predictions.append(output_classes[predicted_label])
                       ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
IndexError: list index out of range


RuntimeError: Execution failed:

  0%|          | 0/201 [00:00<?, ?it/s]
  0%|          | 0/201 [00:02<?, ?it/s]
Traceback (most recent call last):
  File "/workspaces/ISE-challenge/Test/code_runner_9cyxo5ti/project/main.py", line 63, in <module>
    predictions.append(output_classes[predicted_label])
                       ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
IndexError: list index out of range
